# Analisi dei dati raccolti

## Setup

In [1]:
import pandas as pd

In [2]:
def print_data(series: pd.Series) -> None:
    print(series.value_counts())
    print()
    print(f"Massimo: {series.max()}\nMinimo: {series.min()}\nMedia: {series.mean()}\nVarianza: {series.var()}")

## Asconprfav12

In [3]:
path = "../asconprfav12/data/{}.csv"

### Armv7m

In [4]:
with open(path.format("armv7m"), "r") as f:
    aprfa_armv7m = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [5]:
# Generazione tag
data = aprfa_armv7m["0B-A"]

print_data(data)

0B-A
44    956
45     44
Name: count, dtype: int64

Massimo: 45
Minimo: 44
Media: 44.044
Varianza: 0.04210610610610611


In [6]:
# Verifica tag
data = aprfa_armv7m["0B-V"]

print_data(data)

0B-V
95    912
96     88
Name: count, dtype: int64

Massimo: 96
Minimo: 95
Media: 95.088
Varianza: 0.08033633633633637


#### PT 8 byte

In [7]:
# Generazione tag
data = aprfa_armv7m["8B-A"]

print_data(data)

8B-A
44    955
45     45
Name: count, dtype: int64

Massimo: 45
Minimo: 44
Media: 44.045
Varianza: 0.04301801801801802


In [8]:
# Verifica tag
data = aprfa_armv7m["8B-V"]

print_data(data)

8B-V
96    955
97     44
95      1
Name: count, dtype: int64

Massimo: 97
Minimo: 95
Media: 96.043
Varianza: 0.043194194194194194


#### PT 16 byte

In [9]:
# Generazione tag
data = aprfa_armv7m["16B-A"]

print_data(data)

16B-A
45    955
46     45
Name: count, dtype: int64

Massimo: 46
Minimo: 45
Media: 45.045
Varianza: 0.04301801801801803


In [10]:
# Verifica tag
data = aprfa_armv7m["16B-V"]

print_data(data)

16B-V
97    910
98     89
96      1
Name: count, dtype: int64

Massimo: 98
Minimo: 96
Media: 97.088
Varianza: 0.08233833833833837


#### PT 32 byte

In [11]:
# Generazione tag
data = aprfa_armv7m["32B-A"]

print_data(data)

32B-A
46    955
47     45
Name: count, dtype: int64

Massimo: 47
Minimo: 46
Media: 46.045
Varianza: 0.04301801801801802


In [12]:
# Verifica tag
data = aprfa_armv7m["32B-V"]

print_data(data)

32B-V
99     957
100     43
Name: count, dtype: int64

Massimo: 100
Minimo: 99
Media: 99.043
Varianza: 0.04119219219219219


#### PT 64 byte

In [13]:
# Generazione tag
data = aprfa_armv7m["64B-A"]

print_data(data)

64B-A
62    957
63     43
Name: count, dtype: int64

Massimo: 63
Minimo: 62
Media: 62.043
Varianza: 0.041192192192192216


In [14]:
# Verifica tag
data = aprfa_armv7m["64B-V"]

print_data(data)

64B-V
131    867
132    133
Name: count, dtype: int64

Massimo: 132
Minimo: 131
Media: 131.133
Varianza: 0.11542642642642642


#### PT 128 byte

In [15]:
# Generazione tag
data = aprfa_armv7m["128B-A"]

print_data(data)

128B-A
93    912
94     88
Name: count, dtype: int64

Massimo: 94
Minimo: 93
Media: 93.088
Varianza: 0.08033633633633637


In [16]:
# Verifica tag
data = aprfa_armv7m["128B-V"]

print_data(data)

128B-V
194    826
195    173
193      1
Name: count, dtype: int64

Massimo: 195
Minimo: 193
Media: 194.172
Varianza: 0.1445605605605605


#### PT 256 byte

In [17]:
# Generazione tag
data = aprfa_armv7m["256B-A"]

print_data(data)

256B-A
143    871
144    129
Name: count, dtype: int64

Massimo: 144
Minimo: 143
Media: 143.129
Varianza: 0.11247147147147148


In [18]:
# Verifica tag
data = aprfa_armv7m["256B-V"]

print_data(data)

256B-V
292    739
294    261
Name: count, dtype: int64

Massimo: 294
Minimo: 292
Media: 292.522
Varianza: 0.7722882882882883


#### PT 512 byte

In [19]:
# Generazione tag
data = aprfa_armv7m["512B-A"]

print_data(data)

512B-A
241    737
242    262
243      1
Name: count, dtype: int64

Massimo: 243
Minimo: 241
Media: 241.264
Varianza: 0.19650050050050052


In [20]:
# Verifica tag
data = aprfa_armv7m["512B-V"]

print_data(data)

512B-V
490    521
491    478
492      1
Name: count, dtype: int64

Massimo: 492
Minimo: 490
Media: 490.48
Varianza: 0.25185185185185177


#### PT 1024 byte

In [21]:
# Generazione tag
data = aprfa_armv7m["1024B-A"]

print_data(data)

1024B-A
454    480
452    304
453    216
Name: count, dtype: int64

Massimo: 454
Minimo: 452
Media: 453.176
Varianza: 0.7537777777777777


In [22]:
# Verifica tag
data = aprfa_armv7m["1024B-V"]

print_data(data)

1024B-V
913    565
914    349
912     86
Name: count, dtype: int64

Massimo: 914
Minimo: 912
Media: 913.263
Varianza: 0.3661971971971971


### Armv7m_small

In [23]:
with open(path.format("armv7m_small"), "r") as f:
    aprfa_small = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [24]:
# Generazione tag
data = aprfa_small["0B-A"]

print_data(data)

0B-A
38    958
39     41
40      1
Name: count, dtype: int64

Massimo: 40
Minimo: 38
Media: 38.043
Varianza: 0.04319419419419422


In [25]:
# Verifica tag
data = aprfa_small["0B-V"]

print_data(data)

0B-V
80    918
81     80
79      2
Name: count, dtype: int64

Massimo: 81
Minimo: 79
Media: 80.078
Varianza: 0.07599199199199196


#### PT 8 byte

In [26]:
# Generazione tag
data = aprfa_small["8B-A"]

print_data(data)

8B-A
38    957
40     40
39      3
Name: count, dtype: int64

Massimo: 40
Minimo: 38
Media: 38.083
Varianza: 0.15626726726726725


In [27]:
# Verifica tag
data = aprfa_small["8B-V"]

print_data(data)

8B-V
81    918
82     80
80      2
Name: count, dtype: int64

Massimo: 82
Minimo: 80
Media: 81.078
Varianza: 0.07599199199199197


#### PT 16 byte

In [28]:
# Generazione tag
data = aprfa_small["16B-A"]

print_data(data)

16B-A
39    960
40     40
Name: count, dtype: int64

Massimo: 40
Minimo: 39
Media: 39.04
Varianza: 0.038438438438438444


In [29]:
# Verifica tag
data = aprfa_small["16B-V"]

print_data(data)

16B-V
81    920
83     80
Name: count, dtype: int64

Massimo: 83
Minimo: 81
Media: 81.16
Varianza: 0.2946946946946948


#### PT 32 byte

In [30]:
# Generazione tag
data = aprfa_small["32B-A"]

print_data(data)

32B-A
40    960
41     40
Name: count, dtype: int64

Massimo: 41
Minimo: 40
Media: 40.04
Varianza: 0.038438438438438444


In [31]:
# Verifica tag
data = aprfa_small["32B-V"]

print_data(data)

32B-V
83    918
84     81
82      1
Name: count, dtype: int64

Massimo: 84
Minimo: 82
Media: 83.08
Varianza: 0.0756756756756757


#### PT 64 byte

In [32]:
# Generazione tag
data = aprfa_small["64B-A"]

print_data(data)

64B-A
53    960
54     40
Name: count, dtype: int64

Massimo: 54
Minimo: 53
Media: 53.04
Varianza: 0.038438438438438444


In [33]:
# Verifica tag
data = aprfa_small["64B-V"]

print_data(data)

64B-V
110    879
111    120
109      1
Name: count, dtype: int64

Massimo: 111
Minimo: 109
Media: 110.119
Varianza: 0.10694594594594595


#### PT 128 byte

In [34]:
# Generazione tag
data = aprfa_small["128B-A"]

print_data(data)

128B-A
80    919
81     81
Name: count, dtype: int64

Massimo: 81
Minimo: 80
Media: 80.081
Varianza: 0.07451351351351351


In [35]:
# Verifica tag
data = aprfa_small["128B-V"]

print_data(data)

128B-V
163    839
164    161
Name: count, dtype: int64

Massimo: 164
Minimo: 163
Media: 163.161
Varianza: 0.13521421421421423


#### PT 256 byte

In [36]:
# Generazione tag
data = aprfa_small["256B-A"]

print_data(data)

256B-A
122    880
123    120
Name: count, dtype: int64

Massimo: 123
Minimo: 122
Media: 122.12
Varianza: 0.1057057057057057


In [37]:
# Verifica tag
data = aprfa_small["256B-V"]

print_data(data)

256B-V
247    760
248    240
Name: count, dtype: int64

Massimo: 248
Minimo: 247
Media: 247.24
Varianza: 0.18258258258258253


#### PT 512 byte

In [38]:
# Generazione tag
data = aprfa_small["512B-A"]

print_data(data)

512B-A
205    799
206    200
207      1
Name: count, dtype: int64

Massimo: 207
Minimo: 205
Media: 205.202
Varianza: 0.16335935935935933


In [39]:
# Verifica tag
data = aprfa_small["512B-V"]

print_data(data)

512B-V
414    599
416    400
415      1
Name: count, dtype: int64

Massimo: 416
Minimo: 414
Media: 414.801
Varianza: 0.9603593593593593


#### PT 1024 byte

In [40]:
# Generazione tag
data = aprfa_small["1024B-A"]

print_data(data)

1024B-A
384    599
385    400
386      1
Name: count, dtype: int64

Massimo: 386
Minimo: 384
Media: 384.402
Varianza: 0.24263863863863866


In [41]:
# Verifica tag
data = aprfa_small["1024B-V"]

print_data(data)

1024B-V
773    761
772    239
Name: count, dtype: int64

Massimo: 773
Minimo: 772
Media: 772.761
Varianza: 0.182061061061061


### Bi32_armv7m

In [42]:
with open(path.format("bi32_armv7m"), "r") as f:
    aprfa_bi32 = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [43]:
# Generazione tag
data = aprfa_bi32["0B-A"]

print_data(data)

0B-A
36    838
37    122
38     40
Name: count, dtype: int64

Massimo: 38
Minimo: 36
Media: 36.202
Varianza: 0.2414374374374375


In [44]:
# Verifica tag
data = aprfa_bi32["0B-V"]

print_data(data)

0B-V
80    920
81     80
Name: count, dtype: int64

Massimo: 81
Minimo: 80
Media: 80.08
Varianza: 0.0736736736736737


#### PT 8 byte

In [45]:
# Generazione tag
data = aprfa_bi32["8B-A"]

print_data(data)

8B-A
38    957
39     43
Name: count, dtype: int64

Massimo: 39
Minimo: 38
Media: 38.043
Varianza: 0.04119219219219221


In [46]:
# Verifica tag
data = aprfa_bi32["8B-V"]

print_data(data)

8B-V
83    917
84     81
82      2
Name: count, dtype: int64

Massimo: 84
Minimo: 82
Media: 83.079
Varianza: 0.07683583583583582


#### PT 16 byte

In [47]:
# Generazione tag
data = aprfa_bi32["16B-A"]

print_data(data)

16B-A
39    719
40    240
41     41
Name: count, dtype: int64

Massimo: 41
Minimo: 39
Media: 39.322
Varianza: 0.30061661661661665


In [48]:
# Verifica tag
data = aprfa_bi32["16B-V"]

print_data(data)

16B-V
86    918
88     80
87      2
Name: count, dtype: int64

Massimo: 88
Minimo: 86
Media: 86.162
Varianza: 0.29605205205205204


#### PT 32 byte

In [49]:
# Generazione tag
data = aprfa_bi32["32B-A"]

print_data(data)

32B-A
43    960
44     40
Name: count, dtype: int64

Massimo: 44
Minimo: 43
Media: 43.04
Varianza: 0.038438438438438444


In [50]:
# Verifica tag
data = aprfa_bi32["32B-V"]

print_data(data)

32B-V
93    919
94     80
92      1
Name: count, dtype: int64

Massimo: 94
Minimo: 92
Media: 93.079
Varianza: 0.07483383383383382


#### PT 64 byte

In [51]:
# Generazione tag
data = aprfa_bi32["64B-A"]

print_data(data)

64B-A
59    961
60     39
Name: count, dtype: int64

Massimo: 60
Minimo: 59
Media: 59.039
Varianza: 0.03751651651651652


In [52]:
# Verifica tag
data = aprfa_bi32["64B-V"]

print_data(data)

64B-V
124    880
126    120
Name: count, dtype: int64

Massimo: 126
Minimo: 124
Media: 124.24
Varianza: 0.4228228228228228


#### PT 128 byte

In [53]:
# Generazione tag
data = aprfa_bi32["128B-A"]

print_data(data)

128B-A
90    918
91     82
Name: count, dtype: int64

Massimo: 91
Minimo: 90
Media: 90.082
Varianza: 0.07535135135135136


In [54]:
# Verifica tag
data = aprfa_bi32["128B-V"]

print_data(data)

128B-V
188    798
189    201
187      1
Name: count, dtype: int64

Massimo: 189
Minimo: 187
Media: 188.2
Varianza: 0.1621621621621622


#### PT 256 byte

In [55]:
# Generazione tag
data = aprfa_bi32["256B-A"]

print_data(data)

256B-A
144    719
146    161
145    120
Name: count, dtype: int64

Massimo: 146
Minimo: 144
Media: 144.442
Varianza: 0.5692052052052053


In [56]:
# Verifica tag
data = aprfa_bi32["256B-V"]

print_data(data)

256B-V
296    719
297    281
Name: count, dtype: int64

Massimo: 297
Minimo: 296
Media: 296.281
Varianza: 0.2022412412412412


#### PT 512 byte

In [57]:
# Generazione tag
data = aprfa_bi32["512B-A"]

print_data(data)

512B-A
253    759
254    240
252      1
Name: count, dtype: int64

Massimo: 254
Minimo: 252
Media: 253.239
Varianza: 0.18406306306306308


In [58]:
# Verifica tag
data = aprfa_bi32["512B-V"]

print_data(data)

512B-V
514    520
512    479
513      1
Name: count, dtype: int64

Massimo: 514
Minimo: 512
Media: 513.041
Varianza: 0.9983173173173177


#### PT 1024 byte

In [59]:
# Generazione tag
data = aprfa_bi32["1024B-A"]

print_data(data)

1024B-A
478    519
479    479
480      2
Name: count, dtype: int64

Massimo: 480
Minimo: 478
Media: 478.483
Varianza: 0.253964964964965


In [60]:
# Verifica tag
data = aprfa_bi32["1024B-V"]

print_data(data)

1024B-V
965    958
964     40
966      2
Name: count, dtype: int64

Massimo: 966
Minimo: 964
Media: 964.962
Varianza: 0.040596596596596576


### Ref

In [61]:
with open(path.format("ref"), "r") as f:
    aprfa_ref = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [62]:
# Generazione tag
data = aprfa_ref["0B-A"]

print_data(data)

0B-A
134    878
135    121
136      1
Name: count, dtype: int64

Massimo: 136
Minimo: 134
Media: 134.123
Varianza: 0.109980980980981


In [63]:
# Verifica tag
data = aprfa_ref["0B-V"]

print_data(data)

0B-V
271    759
272    240
273      1
Name: count, dtype: int64

Massimo: 273
Minimo: 271
Media: 271.242
Varianza: 0.1856216216216216


#### PT 8 byte

In [64]:
# Generazione tag
data = aprfa_ref["8B-A"]

print_data(data)

8B-A
139    799
141    120
140     81
Name: count, dtype: int64

Massimo: 141
Minimo: 139
Media: 139.321
Varianza: 0.45841741741741737


In [65]:
# Verifica tag
data = aprfa_ref["8B-V"]

print_data(data)

8B-V
281    720
283    278
282      2
Name: count, dtype: int64

Massimo: 283
Minimo: 281
Media: 281.558
Varianza: 0.8034394394394396


#### PT 16 byte

In [66]:
# Generazione tag
data = aprfa_ref["16B-A"]

print_data(data)

16B-A
145    878
146    121
144      1
Name: count, dtype: int64

Massimo: 146
Minimo: 144
Media: 145.12
Varianza: 0.1077077077077077


In [67]:
# Verifica tag
data = aprfa_ref["16B-V"]

print_data(data)

16B-V
292    718
293    280
294      2
Name: count, dtype: int64

Massimo: 294
Minimo: 292
Media: 292.284
Varianza: 0.2075515515515516


#### PT 32 byte

In [68]:
# Generazione tag
data = aprfa_ref["32B-A"]

print_data(data)

32B-A
155    879
156    121
Name: count, dtype: int64

Massimo: 156
Minimo: 155
Media: 155.121
Varianza: 0.10646546546546548


In [69]:
# Verifica tag
data = aprfa_ref["32B-V"]

print_data(data)

32B-V
313    718
314    281
315      1
Name: count, dtype: int64

Massimo: 315
Minimo: 313
Media: 313.283
Varianza: 0.2051161161161162


#### PT 64 byte

In [70]:
# Generazione tag
data = aprfa_ref["64B-A"]

print_data(data)

64B-A
213    601
214    201
215    198
Name: count, dtype: int64

Massimo: 215
Minimo: 213
Media: 213.597
Varianza: 0.6372282282282281


In [71]:
# Verifica tag
data = aprfa_ref["64B-V"]

print_data(data)

64B-V
429    600
431    398
432      1
430      1
Name: count, dtype: int64

Massimo: 432
Minimo: 429
Media: 429.8
Varianza: 0.9629629629629629


#### PT 128 byte

In [72]:
# Generazione tag
data = aprfa_ref["128B-A"]

print_data(data)

128B-A
330    679
331    320
332      1
Name: count, dtype: int64

Massimo: 332
Minimo: 330
Media: 330.322
Varianza: 0.22053653653653651


In [73]:
# Verifica tag
data = aprfa_ref["128B-V"]

print_data(data)

128B-V
664    640
663    358
665      2
Name: count, dtype: int64

Massimo: 665
Minimo: 663
Media: 663.644
Varianza: 0.23349749749749754


#### PT 256 byte

In [74]:
# Generazione tag
data = aprfa_ref["256B-A"]

print_data(data)

256B-A
527    518
526    481
528      1
Name: count, dtype: int64

Massimo: 528
Minimo: 526
Media: 526.52
Varianza: 0.25185185185185177


In [75]:
# Verifica tag
data = aprfa_ref["256B-V"]

print_data(data)

256B-V
1057    959
1058     41
Name: count, dtype: int64

Massimo: 1058
Minimo: 1057
Media: 1057.041
Varianza: 0.039358358358358346


#### PT 512 byte

In [76]:
# Generazione tag
data = aprfa_ref["512B-A"]

print_data(data)

512B-A
920    878
919     81
921     41
Name: count, dtype: int64

Massimo: 921
Minimo: 919
Media: 919.96
Varianza: 0.12052052052052052


In [77]:
# Verifica tag
data = aprfa_ref["512B-V"]

print_data(data)

512B-V
1842    838
1841    160
1843      1
1844      1
Name: count, dtype: int64

Massimo: 1844
Minimo: 1841
Media: 1841.843
Varianza: 0.14049149149149148


#### PT 1024 byte

In [78]:
# Generazione tag
data = aprfa_ref["1024B-A"]

print_data(data)

1024B-A
1743    719
1742    279
1741      1
1744      1
Name: count, dtype: int64

Massimo: 1744
Minimo: 1741
Media: 1742.72
Varianza: 0.2058058058058058


In [79]:
# Verifica tag
data = aprfa_ref["1024B-V"]

print_data(data)

1024B-V
3488    519
3489    480
3491      1
Name: count, dtype: int64

Massimo: 3491
Minimo: 3488
Media: 3488.483
Varianza: 0.25596696696696697
